https://blog.dominodatalab.com/fitting-gaussian-process-models-python/

# Building models with Gaussians

What if we choose to use Gaussian distributions to model our data?

$$p(x|\pi,\Sigma) = (2\pi)^{-k/2}|\Sigma|^{-1/2}\exp{-\frac{1}{2}(x-\mu)'\Sigma^{-1}(x-\mu)}$$

There would not seem to be any gain doing this, because normal dsitributions are not particularly flexible distributions in and of themselves. However, adopting a set of Gaussians (a multivariate normal vector) confers a number of advantages. First, the marginal distribution of any subset of elements from a multivariate normal distribution is also normal:

$$
\begin{eqnarray*}
p(x,y) & = & \mathcal{N}\left(\left[\begin{array}{c}
\mu_x\\
\mu_y
\end{array}\right],\left[\begin{array}{ccc}
\Sigma_x & \Sigma_{xy}\\
\Sigma_{xy}^T & \Sigma_y
\end{array}\right]\right)
\end{eqnarray*}
$$

$$p(x) = \int p(x,y) dy = \mathcal{N}(\mu_x, \Sigma_x)$$